In [1]:
import copy
import math
import random
import time
from pathlib import Path
import os
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.init as init
from IPython.display import Audio, display
from torch import optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm

In [2]:
seed = 0
random.seed(seed)

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

device

device(type='cpu')

## Load the dataset
Data can be downloaded from http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz

In [3]:
from helpers.utils import load_from_path

data_path = Path('data/speech_commands_v0.01')
train_, valid_, test_ = load_from_path(data_path)

print(f'training list has {len(train_)} elements')
print(f'testing list has {len(test_)} elements')
print(f'validation list has {len(valid_)} elements')

training list has 51088 elements
testing list has 6835 elements
validation list has 6798 elements


In [4]:
background_noise_list = []
for root, _, files in os.walk(data_path / '_background_noise_'):
    root_path = Path(root)
    for file in files:
        if not file.endswith('.wav'):
            continue
        data_entry = {
            'file_path': root_path / file,
            'label': file[:-4]
        }
        background_noise_list.append(data_entry)
print(f'background noise files: {[noise['label'] for noise in background_noise_list]}')

background noise files: ['running_tap', 'doing_the_dishes', 'pink_noise', 'exercise_bike', 'white_noise', 'dude_miaowing']


## Use dataset builder

In [5]:
from helpers.dataset_builder import DatasetBuilder

builder = DatasetBuilder(train_, valid_, test_, seed)
train, valid, test = builder \
    .keep_labels(['yes', 'no'], make_rest_unknown=False) \
    .build()

label2id = builder.label2id
id2label = builder.id2label

train: 3713  | {'yes': 1860, 'no': 1853}
  val:  531  | {'yes': 261, 'no': 270}
 test:  508  | {'yes': 256, 'no': 252}


### Data visualization – audio examples

In [6]:
from helpers.utils import show_random_example_per_class

show_random_example_per_class(train, 2)

no


yes


In [7]:
from helpers.utils import preprocess_dataset

X_train, y_train = preprocess_dataset(train, "train")
X_val,   y_val   = preprocess_dataset(valid, "valid")
X_test,  y_test  = preprocess_dataset(test,  "test")

print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

Pre-processing test: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 508/508 [00:00<00:00, 625.76it/s]


Train shape: torch.Size([3713, 40, 101]) torch.Size([3713])
Validation shape: torch.Size([531, 40, 101]) torch.Size([531])
Test shape: torch.Size([508, 40, 101]) torch.Size([508])


## Models and training

In [8]:
batch_size = 64

train_ds = TensorDataset(X_train, y_train)
val_ds   = TensorDataset(X_val,   y_val)
test_ds  = TensorDataset(X_test,  y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

print(f"{len(train_loader)=}, {len(val_loader)=}, {len(test_loader)=}")

len(train_loader)=58, len(val_loader)=9, len(test_loader)=8


### Baseline CNN

In [9]:
from helpers.utils import train_model, eval_model

In [11]:
from helpers.models import ConvBaseline

cnn_model  = ConvBaseline(len(id2label)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=3e-4)
scheduler = StepLR(optimizer, step_size=6, gamma=0.5)

epochs = 40
history, best_val = train_model(
    cnn_model,
    device,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler=scheduler,
    epochs=epochs,
)
print(f"Best val-accuracy: {best_val:.3%}")

[01/40] train 0.3382/0.877 – val 0.2369/0.925
[02/40] train 0.1605/0.953 – val 0.1154/0.960
[03/40] train 0.1026/0.972 – val 0.0811/0.968
[04/40] train 0.0828/0.976 – val 0.0733/0.974
[05/40] train 0.0683/0.979 – val 0.0560/0.974
[06/40] train 0.0577/0.982 – val 0.0748/0.966
[07/40] train 0.0473/0.985 – val 0.0566/0.979
[08/40] train 0.0413/0.988 – val 0.0450/0.981
[09/40] train 0.0400/0.986 – val 0.0463/0.985
[10/40] train 0.0354/0.988 – val 0.0435/0.981
[11/40] train 0.0330/0.989 – val 0.0525/0.979
[12/40] train 0.0281/0.991 – val 0.0426/0.985
⏹️  Early stopping after 12 epochs (no val-improve for 3 epochs).
Best val-accuracy: 98.493%


In [12]:
eval_model(device, cnn_model, test_loader, 'Baseline CNN', id2label)

Test accuracy: 96.850%


### Transformer

In [13]:
from helpers.models import TransformerClassifier

tfm_model = TransformerClassifier(
    len(id2label),
    n_mfcc=40,
    d_model=128,
    n_heads=4,
    num_layers=4,
    dim_ff=256,
    dropout=0.1,
    pool="cls"             # or "mean"
).to(device)

criterion  = nn.CrossEntropyLoss()
optimizer  = optim.AdamW(tfm_model.parameters(), lr=2e-4, weight_decay=1e-2)
scheduler  = StepLR(optimizer, step_size=8, gamma=0.5)

In [14]:
epochs = 40
history, best_val = train_model(
    tfm_model,
    device,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler=scheduler,
    epochs=epochs,
)
print(f"Best val-accuracy: {best_val:.3%}")

[01/40] train 0.2903/0.873 – val 0.1263/0.953
[02/40] train 0.1373/0.949 – val 0.0990/0.964
[03/40] train 0.1135/0.956 – val 0.0648/0.976
[04/40] train 0.0948/0.963 – val 0.0734/0.972
[05/40] train 0.0611/0.975 – val 0.0995/0.960
[06/40] train 0.0597/0.977 – val 0.0625/0.976
⏹️  Early stopping after 6 epochs (no val-improve for 3 epochs).
Best val-accuracy: 97.552%


In [15]:
eval_model(device, tfm_model, test_loader, 'Transformer', id2label)

Test accuracy: 96.260%
